# Week 2 assignment
https://www.coursera.org/learn/dsp/exam/jYkyi/homework-for-module-2

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys 

path = 'C:\\Users\\Wei\\Dropbox\\Coding\\notebooks\\專案\\待處理\\Waves\\bitbucket\\github\\codes'
path = '/home/wei/Dropbox/Coding/notebooks/專案/待處理/Waves/bitbucket/github/codes'
sys.path.append(path)

from waves.tools.wavelet import *

### 2. Find v3 to form basis

In [3]:
v0 = np.array(( 1,  1,  1,  1)).reshape((-1, 1))
v1 = np.array(( 1,  1, -1, -1)).reshape((-1, 1))
v2 = np.array(( 1, -1,  1, -1)).reshape((-1, 1))

np.hstack((v0, v1, v2))

array([[ 1,  1,  1],
       [ 1,  1, -1],
       [ 1, -1,  1],
       [ 1, -1, -1]])

### 列出適合的 v3

In [4]:
from itertools import permutations, combinations


def list_v3s():

    def count(a, n):
        return a.count(n)
           
    v3s = []
    perms = permutations((-1, 1, 0) * 4, 4)  

    for perm in perms:
        v3 =  np.array(perm).reshape((-1, 1))
        V = np.hstack((v0, v1, v2, v3))
        d = np.linalg.det(V)

        if  d != 0:
            v3s.append((count(perm, 0), *perm))  

    v3s = np.array(sorted(v3s))
    v3s = np.unique(v3s, axis = 0)
    
    return v3s

v3s = list_v3s()
v3s[:11]

array([[ 0, -1, -1, -1,  1],
       [ 0, -1, -1,  1, -1],
       [ 0, -1,  1, -1, -1],
       [ 0, -1,  1,  1, -1],
       [ 0, -1,  1,  1,  1],
       [ 0,  1, -1, -1, -1],
       [ 0,  1, -1, -1,  1],
       [ 0,  1, -1,  1,  1],
       [ 0,  1,  1, -1,  1],
       [ 0,  1,  1,  1, -1],
       [ 1, -1, -1, -1,  0]])

### 求取能讓 V 有 invert 矩陣的 v3

In [5]:
def has_inv(v3s):
    
    matchs = []
    
    for v3 in v3s: 
        
        V = np.hstack((v0, v1, v2, v3[1:].reshape((-1, 1))))
        
        try:
            invert = np.dot(V, np.linalg.inv(V))            
            matchs.append(v3[1:])
            
        except:
            pass
        
    return np.vstack(matchs)

matchs = has_inv(v3s)
matchs[:10]

array([[-1, -1, -1,  1],
       [-1, -1,  1, -1],
       [-1,  1, -1, -1],
       [-1,  1,  1, -1],
       [-1,  1,  1,  1],
       [ 1, -1, -1, -1],
       [ 1, -1, -1,  1],
       [ 1, -1,  1,  1],
       [ 1,  1, -1,  1],
       [ 1,  1,  1, -1]])

### 驗證

In [6]:
def verify(matchs, v0 = v0, v1 = v1, v2 = v2):
    
    def check(v3, v0 = v0, v1 = v1, v2 = v2):
        v3 = v3.reshape((-1, 1))

        v0 = v0 / np.linalg.norm(v0)
        v1 = v1 / np.linalg.norm(v1)
        v2 = v2 / np.linalg.norm(v2)
        v3 = v3 / np.linalg.norm(v3)

        V = np.hstack((v0, v1, v2, v3))

        if np.linalg.det(V) == 0:
            return False

        try:
            inv = np.dot(V, np.linalg.inv(V))        
        except:
            return False

        return True

        
    verifieds = []

    for match in matchs: 
        if check(match):
            verifieds.append(match)
            
    return np.vstack(verifieds)
        
verifieds = verify(matchs)
verifieds[:11]

array([[-1, -1, -1,  1],
       [-1, -1,  1, -1],
       [-1,  1, -1, -1],
       [-1,  1,  1, -1],
       [-1,  1,  1,  1],
       [ 1, -1, -1, -1],
       [ 1, -1, -1,  1],
       [ 1, -1,  1,  1],
       [ 1,  1, -1,  1],
       [ 1,  1,  1, -1],
       [-1, -1, -1,  0]])

### 任意選定 v3 來驗證

In [7]:
v3 = verifieds[3]
v3 = np.array(v3).reshape((-1, 1))

In [8]:
v0 = v0 / np.linalg.norm(v0)
v1 = v1 / np.linalg.norm(v1)
v2 = v2 / np.linalg.norm(v2)
v3 = v3 / np.linalg.norm(v3)

V = np.hstack((v0, v1, v2, v3))
V

array([[ 0.5,  0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5,  0.5],
       [ 0.5, -0.5, -0.5, -0.5]])

### 檢查是否 det 不為 0

In [9]:
np.linalg.det(V)

1.0

### 檢查是否有反矩陣

In [10]:
np.dot(V, np.linalg.inv(V)) 

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [11]:
np.dot(np.linalg.inv(V), V) 

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

### 3. 

In [13]:
y = np.array((1, 2, -1, 0)).reshape((-1, 1))
y

array([[ 1],
       [ 2],
       [-1],
       [ 0]])

In [14]:
x = np.dot(np.linalg.inv(V), y)
print(str(x.ravel().tolist()).replace(',', ''))

[1.0 2.0 -1.0 0.0]


In [16]:
np.all(np.dot(V, x) == y)

True

### 對每一個 v3，y 在 V 空間中的座標是唯一

In [15]:
def get_x(V):    
    x = np.dot(np.linalg.inv(V), y)
    print(str(x.ravel().tolist()).replace(',', ''))
    
    
for verified in verifieds:
    
    v3 =  verified.reshape((-1, 1)) 
    
    v0 = v0 / np.linalg.norm(v0)
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)
    v3 = v3 / np.linalg.norm(v3)

    V = np.hstack((v0, v1, v2, v3))
    
    get_x(V)

[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0000000000000004 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0000000000000004 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[1.0 2.0 -1.0 0.0]
[

### 5. 
1 0 -1 -1 1 0 0 -1 1

In [5]:
D = np.array([[1, 0, -1],[-1, 1, 0],[0, -1, 1]])
D

array([[ 1,  0, -1],
       [-1,  1,  0],
       [ 0, -1,  1]])

In [6]:
str(D.ravel().tolist()).replace(',', '')

'[1 0 -1 -1 1 0 0 -1 1]'

### 6. 
1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1

In [15]:
a = np.array((0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0)).reshape((4, 4))
a

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]])

In [16]:
a4 = np.linalg.matrix_power(a, 4)
a4

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [17]:
str(a4.ravel().tolist()).replace(',', '')

'[1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1]'